In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import swifter
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import graphviz
from sklearn import tree

In [2]:
# code for exporting the information as pickle format.
# info.to_pickle('data_format1/user_info_format1.pkl')
# log = log.to_pickle('data_format1/user_log_format1.pkl')

In [3]:
# code for importing the original data.
info = pd.read_csv('data_format1/user_info_format1.csv')
log = pd.read_csv('data_format1/user_log_format1.csv')
df_train = pd.read_csv('data_format1/train_format1.csv')

In [4]:
df_Train = pd.read_csv('data_format1/train_format1.csv')

In [5]:
# info_ori = info.copy()
# log_ori = log.copy()
# df_train_ori = df_train.copy()

In [6]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    424170 non-null  int64  
 1   age_range  421953 non-null  float64
 2   gender     417734 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 9.7 MB


In [7]:
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54925330 entries, 0 to 54925329
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      int64  
 1   item_id      int64  
 2   cat_id       int64  
 3   seller_id    int64  
 4   brand_id     float64
 5   time_stamp   int64  
 6   action_type  int64  
dtypes: float64(1), int64(6)
memory usage: 2.9 GB


In [8]:
info.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


In [9]:
log.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


In [10]:
info['age_range'] = info['age_range'].fillna(0)
info['gender'] = info['gender'].fillna(2)

In [11]:
info['age_range'] = info['age_range'].astype('category')
info['gender'] = info['gender'].astype('category')

In [12]:
info['age_range'] = info['age_range'].replace(8, 7)

In [13]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   user_id    424170 non-null  int64   
 1   age_range  424170 non-null  category
 2   gender     424170 non-null  category
dtypes: category(2), int64(1)
memory usage: 4.0 MB


In [14]:
def ChangeDateFormat(date):
    month = date // 100
    day = date % 100
    date = '2020-' + str(month) + '-' + str(day)
    return pd.Timestamp(date)

In [16]:
log['time_stamp'] = log['time_stamp'].apply(lambda x: ChangeDateFormat(x))

KeyboardInterrupt: 

In [15]:
# log['time_stamp'] = log['time_stamp'].swifter.apply(lambda x: ChangeDateFormat(x))

KeyboardInterrupt: 

In [ ]:
log['time_stamp'].value_counts().plot(kind='line')

In [ ]:
fig = info['age_range'].value_counts(sort=False).plot(kind='barh')
fig.set_yticklabels(["unknown","<18","18-24","25-29","30-34","35-39","40-49",">=50"])

In [ ]:
fig = info['gender'].value_counts(sort=False).plot(kind='barh')
fig.set_yticklabels(["female","male", "unknown"])

In [ ]:
fig = log['action_type'].value_counts(sort=False).plot(kind='barh')
fig.set_yticklabels(['click', 'add-to-cart', 'purchase', 'add-to-favorite'])

In [ ]:
ct = pd.crosstab(info['age_range'], info['gender'], margins=True)
fig = ct.iloc[:,0:-1].div(ct.iloc[:,-1], axis=0).round(3).iloc[0:-1,:].plot(kind='bar')
fig.set_xticklabels(["unknown","<18","18-24","25-29","30-34","35-39","40-49",">=50"])
fig.set_xlabel('Age Group')
fig.set_ylabel('Frequncy')
plt.legend(['female', 'male', 'unknown'])

### Seller info

In [ ]:
seller_item_count = log[['seller_id','item_id']]
seller_item_count = seller_item_count.drop_duplicates()
seller_item_count['seller_item_count'] = 1
seller_item_count = seller_item_count.groupby(['seller_id']).agg('sum')
seller_item_count = seller_item_count.drop(columns=['item_id'])
seller_item_count.head()

In [ ]:
seller_brand_count = log[['seller_id','brand_id']]
seller_brand_count = seller_brand_count.drop_duplicates()
seller_brand_count = seller_brand_count.groupby(['seller_id']).agg(len)
seller_brand_count.rename(columns = {"brand_id": "brand_count"},inplace=True) 
seller_brand_count.head()

In [ ]:
item_info = log['item_id'].value_counts()
cat_info = log['cat_id'].value_counts()
brand_info = log['brand_id'].value_counts()
seller_info = log['seller_id'].value_counts()
print(item_info.head(), item_info.size)
print(cat_info.head(),cat_info.size)
print(brand_info.head(),brand_info.size)
print(seller_info.head(),seller_info.size)

In [ ]:
log.head()

In [ ]:
log_sample = log.sample(frac=0.005,random_state=1)

In [ ]:
# log_sample.pivot_table(index=['user_id','seller_id'],aggfunc=len)

In [ ]:
user_seller = log_sample.groupby(['user_id','seller_id'])['item_id'].count()

In [ ]:
user_seller.head()

In [ ]:
user_seller[user_seller > 1]

In [ ]:
user_seller = user_seller > 1

In [ ]:
user_seller.head()

In [ ]:
df_train.head()

### user action

In [17]:
actions = log.action_type
actions = pd.get_dummies(actions,prefix='action_count')
user_action = pd.concat([log.user_id,actions],axis=1)
# user_action = log[['user_id','action_type']]
# user_action.action_type = user_action.action_type.astype('category')

user_action = user_action.groupby('user_id').agg('sum')
user_action.head()

,action_count_0,action_count_1,action_count_2,action_count_3
user_id,,,,
1,27.0,0.0,6.0,0.0
2,47.0,0.0,14.0,2.0
3,63.0,0.0,4.0,1.0
4,49.0,0.0,1.0,0.0
5,150.0,0.0,13.0,10.0


In [ ]:
user_action.boxplot()

In [ ]:
df_Train.head(3)

In [ ]:
# df_label1 = df_Train[df_Train.label==1]
# df_label11 = df_label1.copy()

In [ ]:
# for i in range(10):
#     df_label11 = pd.concat([df_label11,df_label1],axis=0)
#     print(df_label11.shape[0])    

In [ ]:
# df_train = pd.concat([df_Train,df_label11],axis=0)

In [ ]:
seller = seller_item_count.merge(seller_brand_count,on='seller_id')

In [ ]:
df_train = df_Train.merge(seller,left_on='merchant_id',right_on='seller_id')

In [ ]:
df_train = df_train.merge(user_action,on='user_id')

In [ ]:
df_train.head(3)

In [ ]:
log_dummy = pd.get_dummies(log['action_type'],prefix='action')
log_dummy = pd.concat([log.user_id,log.seller_id,actions],axis=1)
log_dummy.columns = ['user_id', 'merchant_id', 'action_0', 'action_1',
       'action_2', 'action_3']
log_dummy = log_dummy.groupby(['user_id','merchant_id']).agg('sum')

In [ ]:
log_dummy.head(3)

In [ ]:
df_train = df_train.merge(log_dummy,on=['merchant_id','user_id'])

In [ ]:
df_train.head(3)

In [ ]:
train,test = train_test_split(df_train,test_size=0.2,random_state=0,shuffle=True,stratify=df_train.label)

In [ ]:
train.head(3)

In [ ]:
targets_train = train.label
targets_test = test.label
attributes_train = train.drop(['merchant_id','user_id','label'],axis=1)
attributes_test = test.drop(['merchant_id','user_id','label'],axis=1)

In [ ]:
attributes_train.head(3)

## Logistic Regression

In [ ]:
lgr = LogisticRegression(max_iter=1000)
hparam = {'C':[0.01,0.1,1,10,100]}
lgrCV = GridSearchCV(lgr,param_grid=hparam)
lgrCV.fit(attributes_train,targets_train)
results = pd.DataFrame()
results['hparam'] = hparam['C']
results['error'] = 1 - lgrCV.cv_results_['mean_test_score']
results.plot.line(x='hparam',y='error')

In [ ]:
(lgrCV.best_estimator_.predict(attributes_test) == targets_test).mean()

## Decision Tree

In [ ]:
hparam = {'max_depth':range(1,6)}

In [ ]:
Tree = DecisionTreeClassifier(random_state=1)
treeCV = GridSearchCV(Tree, param_grid=hparam, return_train_score=True)

In [ ]:
treeCV.fit(attributes_train, targets_train)
print('best param', treeCV.best_params_)
print("test errror rate", (1 - treeCV.best_score_).round(3))

In [ ]:
(treeCV.best_estimator_.predict(attributes_test) == targets_test).mean()

In [ ]:
1-3190/48983

## Bayesian

In [ ]:
hparam = {'alpha':np.arange(1,10)/10}
NBayes = MultinomialNB()
NBayesCV = GridSearchCV(NBayes,param_grid = hparam)
NBayesCV.fit(attributes_train,targets_train)

In [ ]:
NBayesCV.best_params_

In [ ]:
(NBayesCV.best_estimator_.predict(attributes_test) == targets_test).mean()

In [ ]:
df_train.to_pickle('data_format1/df_train_good.pkl')